# Shingle Dependency Trees for datasketch's Minhash

In [ ]:
import sys
sys.path.append("..")

In [2]:
import conllu
import treesimi as ts
import datasketch
import json
import itertools

## Load Dataset

In [3]:
%%capture
!mkdir "../data"
!wget -O "../data/de_hdt-ud-dev.conllu" "https://raw.githubusercontent.com/UniversalDependencies/UD_German-HDT/master/de_hdt-ud-dev.conllu"

In [4]:
dat = conllu.parse(open("../data/de_hdt-ud-dev.conllu").read())
len(dat)

18434

# Generate Shingles and Build LSH
- `MinHashLSH` -- LSH is updatable, Threshold is fixed, Num results unknown (Redis support)
- `MinHashLSHForest` -- LSH is updatable, Threshold unknown, Top-N are returned
- `MinHashLSHEnsemble` -- LSH is fixed, Threshold is fixed (Redis support)

In [5]:
%%time

cfg = {
    'use_trunc_leaves': False,
    'use_drop_nodes': False,
    'use_replace_attr': False
}

# Instantiate LSH Forest
lsh1 = datasketch.MinHashLSH(num_perm=256, threshold=0.8)
lsh2 = datasketch.MinHashLSHForest(num_perm=256)
lsh3 = datasketch.MinHashLSHEnsemble(num_perm=256, threshold=0.8, num_part=32)
lsh3examples = []

mhash = []
for i in range(len(dat)):
    # Read tree data 
    adjac = [(t['id'], t['head'], t['deprel']) for t in dat[i]]
    nested = ts.adjac_to_nested_with_attr(adjac)
    nested = ts.remove_node_ids(nested)
    
    # Shingling
    shingled = ts.shingleset(nested, **cfg)
    stringified = [json.dumps(tree).encode('utf-8') for tree in shingled]

    # Create MinHash objects
    m = datasketch.MinHash(num_perm=256)
    for s in stringified:
        m.update(s)
    
    # Add to LSH
    lsh1.insert(i, m)
    lsh2.add(i, m)
    lsh3examples.append((i, m, len(stringified)))
    
    # save objects
    mhash.append(m)

# Call index method for LSH Forest
lsh2.index()
# Call index method to build the whole LSH Ensemble
lsh3.index(tuple(lsh3examples))

CPU times: user 2min 11s, sys: 1.68 s, total: 2min 13s
Wall time: 2min 37s


# Query for the top-N results

## Just LSH
Finds just itself.

In [21]:
"""
j = 1
res = lsh1.query(mhash[j])

print(f"#num results: {len(res)}")
for i in res:
    print(f"{i:>8d}: {dat[i].metadata['text']}")
"""
None

In [30]:
# try to find any clusters of similar sentences
results = []
for j in range(len(mhash)):
    results.append(lsh1.query(mhash[j]))

In [63]:
# number of examples with at least 1 similar example
tmp = [res for res in results if len(res) >= 2]
n_examples = set(itertools.chain(*tmp))
print(len(n_examples))

2757
#num results: 2
      48 | 1.0000 | Toysmart.com hatte seinen Kunden zugesichert , dass ihre Daten nicht an Dritte weitergegeben würden .
    4760 | 0.6836 | Diese Zahl muss man aber per Telefon an die Sendezentrale durchgeben , damit dort der Gewinner ermittelt werden kann .


In [67]:
res = tmp[8]
print(f"#num results: {len(res)}")
j = res[0]
for i in res:
    print(f"{i:>8d} | {mhash[i].jaccard(mhash[j]):6.4f} | {dat[i].metadata['text']}")

#num results: 2
      60 | 1.0000 | Im Vergleich zum Vorjahr ist die Zahl der deutschen Internet-Teilnehmer um rund 60 Prozent gestiegen .
    7286 | 0.6211 | Etwa 95 Prozent des innerdeutschen E-Mail-Austauschs werde heute über DE-CIX in Frankfurt am Main abgewickelt .


## Top-N Results (LSH Forest)

In [69]:
j = 1
res = lsh2.query(mhash[j], 5)
print(f"#num results: {len(res)}")
for i in res:
    #print(f"{i:>8d}: {dat[i].metadata['text']}")
    print(f"{i:>8d} | {mhash[i].jaccard(mhash[j]):6.4f} | {dat[i].metadata['text']}")

#num results: 5
       1 | 1.0000 | Begleitet von Marktgerüchten über den bevorstehenden Konkurs von Amazon , setzt die Aktie des Online-Händlers ihre Talfahrt fort .
     486 | 0.2695 | Man habe das eCash-Verfahren in Zusammenarbeit mit DigiCash erfolgreich weiterentwickelt .
    8712 | 0.3203 | Die Prüflinge in Bayern müssen die Prüfung am 21. Juni erneut ablegen .
   16017 | 0.2812 | Der Besuch beim Zahnarzt könnte bald seinen Schrecken verlieren .
   10258 | 0.3359 | Der Internethandel mit Wertpapieren hat in letzter Zeit einen Boom erlebt .


## Containment Query (LSH Ensemble)

In [71]:
j = 2
res = [key for key in lsh3.query(lsh3examples[j][1], lsh3examples[j][2])]
print(f"#num results: {len(res)}")
for i in res:
    #print(f"{i:>8d}: {dat[i].metadata['text']}")
    print(f"{i:>8d} | {mhash[i].jaccard(mhash[j]):6.4f} | {dat[i].metadata['text']}")

#num results: 946
    9216 | 0.2070 | In Europa wurde 6,2 Prozent weniger verkauft , in den USA 3,3 Prozent .
    9217 | 0.3672 | Nur im asiatischen Pazifikraum erzielte die Branche einen Wachstum von 2 Prozent .
       3 | 0.3672 | Im Januar hatte die Aktie noch einen Höchststand von 24,50 Euro erreicht .
   12311 | 0.2656 | In elektrische Signale umgewandelt , lassen sich die minimalen Widerstandsänderungen zur Tonübertragung nutzen .
   15895 | 0.3984 | Ab kommenden Mittwoch verkauft Aldi-Nord einen neuen Medion-PC für 2298 Mark ( ohne Monitor ) .
    5145 | 0.2656 | Der Wertpapierhandel an deutschen Präsenzbörsen wird von 17.30 auf 20 Uhr ausgedehnt .
   11291 | 0.2539 | Beide Unternehmen erklärten übereinstimmend , dies sei wegen Finanzierungsschwierigkeiten nicht vollzogen worden .
    7710 | 0.2227 | Unter Myers wurden die Aufgaben des Space Command für den Informationskrieg massiv ausgebaut .
     549 | 0.2969 | Damit ist man vor dem ShareSniffer und ähnlichen Port-Scannern sic